In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
class Conv_op:
  # A Convolution layer using 3x3 filters.

  def __init__(self, num_filters):
    self.num_filters = num_filters

    self.filters = np.random.randn(num_filters, 3, 3) / 9

  def iterate_regions(self, image):
    
    h, w = image.shape

    for i in range(h - 2):
      for j in range(w - 2):
        im_region = image[i:(i + 3), j:(j + 3)]
        yield im_region, i, j

  def forward(self, input):
    
    self.last_input = input

    h, w = input.shape
    output = np.zeros((h - 2, w - 2, self.num_filters))

    for im_region, i, j in self.iterate_regions(input):
      output[i, j] = np.sum(im_region * self.filters, axis=(1, 2))

    return output

  def backprop(self, d_L_d_out, learn_rate):
    
    d_L_d_filters = np.zeros(self.filters.shape)

    for im_region, i, j in self.iterate_regions(self.last_input):
      for f in range(self.num_filters):
        d_L_d_filters[f] += d_L_d_out[i, j, f] * im_region

    
    self.filters -= learn_rate * d_L_d_filters

    return None

In [4]:

class Max_pool:
  # A Max Pooling layer using a pool size of 2.

  def iterate_regions(self, image):
    
    h, w, _ = image.shape
    new_h = h // 2
    new_w = w // 2

    for i in range(new_h):
      for j in range(new_w):
        im_region = image[(i * 2):(i * 2 + 2), (j * 2):(j * 2 + 2)]
        yield im_region, i, j

  def forward(self, input):
   
    self.last_input = input

    h, w, num_filters = input.shape
    output = np.zeros((h // 2, w // 2, num_filters))

    for im_region, i, j in self.iterate_regions(input):
      output[i, j] = np.amax(im_region, axis=(0, 1))

    return output

  def backprop(self, d_L_d_out):
    
    d_L_d_input = np.zeros(self.last_input.shape)

    for im_region, i, j in self.iterate_regions(self.last_input):
      h, w, f = im_region.shape
      amax = np.amax(im_region, axis=(0, 1))

      for i2 in range(h):
        for j2 in range(w):
          for f2 in range(f):
            # If this pixel was the max value, copy the gradient to it.
            if im_region[i2, j2, f2] == amax[f2]:
              d_L_d_input[i * 2 + i2, j * 2 + j2, f2] = d_L_d_out[i, j, f2]

    return d_L_d_input

In [5]:
class Softmax:

    def __init__(self, input_len, nodes):
        self.weights = np.random.randn(input_len, nodes) / input_len
        self.biases = np.zeros(nodes)

    def forward(self, input):
        self.last_input_shape = input.shape
        
        input = input.flatten()
        self.last_input = input

        input_len, nodes = self.weights.shape

        totals = np.dot(input, self.weights) + self.biases
        self.last_totals = totals
 
        exp = np.exp(totals)
        return exp / np.sum(exp, axis=0)
    
    def backprop(self, d_L_d_out, learn_rate):
    # We know only 1 element of d_L_d_out will be nonzero
         for i, gradient in enumerate(d_L_d_out):
            if gradient == 0:
                continue
            t_exp = np.exp(self.last_totals)

      # Sum of all e^totals
            S = np.sum(t_exp)

      # Gradients of out[i] against totals
            d_out_d_t = -t_exp[i] * t_exp / (S ** 2)
            d_out_d_t[i] = t_exp[i] * (S - t_exp[i]) / (S ** 2)

      # Gradients of totals against weights/biases/input
            d_t_d_w = self.last_input
            d_t_d_b = 1
            d_t_d_inputs = self.weights

      # Gradients of loss against totals
            d_L_d_t = gradient * d_out_d_t

      # Gradients of loss against weights/biases/input
            d_L_d_w = d_t_d_w[np.newaxis].T @ d_L_d_t[np.newaxis]
            d_L_d_b = d_L_d_t * d_t_d_b
            d_L_d_inputs = d_t_d_inputs @ d_L_d_t

      # Update weights / biases
            self.weights -= learn_rate * d_L_d_w
            self.biases -= learn_rate * d_L_d_b

            return d_L_d_inputs.reshape(self.last_input_shape)


In [6]:
image_path=r'C:\Users\STACBD\bangla-money\Training\1\1_0.jpg'
a = plt.imread(image_path)
a = np.array(a)
a.shape

(120, 250, 3)

In [7]:
def rgb2gray(rgb):
    return np.dot(rgb[..., :3], [0.299, 0.587, 0.144])

In [8]:
X = rgb2gray(a)
X.shape

(120, 250)

In [9]:
conn = Conv_op(8)
out = conn.forward(X)
out.shape

(118, 248, 8)

In [10]:
pool = Max_pool()  
out2 = pool.forward(out)
out2.shape

(59, 124, 8)

In [11]:
soft=Softmax(59* 124* 8, 9)
out3=soft.forward(out2)
print(out3)

[0.10181045 0.09681898 0.08962254 0.12933633 0.11885105 0.09979783
 0.15502615 0.11782091 0.09091576]


In [12]:
folder = []


def img_read(number, loop):
    for i in range(loop):
        a = plt.imread(
            "C:\\Users\\STACBD\\bangla-money\\Training\\"
            + str(number)
            + "\\"
            + str(number)
            + "_"
            + str(i)
            + ".jpg"
        )
        a = np.array(a)
        folder.append(a)
    ax = np.array(folder)
    return a

In [13]:
y = []


def label(number, loop):
    for i in range(loop):
        if number == 1:
            y.append(0)
        elif number == 2:
            y.append(1)
        elif number == 5:
            y.append(2)
        elif number == 10:
            y.append(3)
        elif number == 20:
            y.append(4)
        elif number == 50:
            y.append(5)
        elif number == 100:
            y.append(6)
        elif number == 500:
            y.append(7)
        elif number == 1000:
            y.append(8)
    return y

In [14]:
one_taka = img_read(1, 100)
one = label(1, 100)
two_taka = img_read(2, 212)
two = label(2, 212)
five_taka = img_read(5, 212)
five = label(5, 212)
ten_taka = img_read(10, 212)
ten = label(10, 212)
fifty_taka = img_read(50, 212)
fifty = label(50, 212)
hundred_taka = img_read(100, 203)
hundred = label(100, 203)
five_hundred_taka = img_read(500, 135)
five_hundred = label(500, 135)
one_thousand_taka = img_read(1000, 166)
one_thousand = label(1000, 166)

In [15]:
y = np.array(y)
print(y)
y.shape

[0 0 0 ... 8 8 8]


(1452,)

In [16]:
folder = np.array(folder)
folder.shape
X = np.array(folder)

In [17]:
def rgb2gray(rgb):
    return np.dot(rgb[..., :3], [0.299, 0.587, 0.144])

In [18]:
X = rgb2gray(folder)

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42, test_size=0.2
)

In [21]:
def forward(image, label):
  out = conn.forward((image / 255) - 0.5)
  out = pool.forward(out)
  out = soft.forward(out)
  loss = -np.log(out[label])
  acc = 1 if np.argmax(out) == label else 0
    
  return out, loss, acc

In [22]:
loss = 0
num_correct = 0
print("ForwardPass")
for i, (im, label) in enumerate(zip(X_train, y_train)):
    _, l, acc = forward(im, label)
    loss += l
    num_correct += acc

    # Print starts every 100 steps.
    if i % 100 == 99:
        print(
            "[Step %d] : Loss %.2f && Accuracy: %d%%" % (i + 1, loss / 100, num_correct)
        )
        loss = 0
        num_correct = 0

ForwardPass
[Step 100] : Loss 2.20 && Accuracy: 16%
[Step 200] : Loss 2.20 && Accuracy: 8%
[Step 300] : Loss 2.20 && Accuracy: 13%
[Step 400] : Loss 2.20 && Accuracy: 14%
[Step 500] : Loss 2.20 && Accuracy: 13%
[Step 600] : Loss 2.20 && Accuracy: 10%
[Step 700] : Loss 2.20 && Accuracy: 11%
[Step 800] : Loss 2.20 && Accuracy: 8%
[Step 900] : Loss 2.20 && Accuracy: 6%
[Step 1000] : Loss 2.20 && Accuracy: 7%
[Step 1100] : Loss 2.20 && Accuracy: 12%


In [23]:
def train(im, label, lr=.005):
  
  # Forward
  out, loss, acc = forward(im, label)

  # Calculate initial gradient
  gradient = np.zeros(10)
  gradient[label] = -1 / out[label]

  # Backprop
  gradient = soft.backprop(gradient, lr)
  gradient = pool.backprop(gradient)
  gradient = conn.backprop(gradient, lr)

  return loss, acc

print("BackPropagation")
loss = 0
num_correct = 0
for i, (im, label) in enumerate(zip(X_train,y_train)):
  if i % 100 == 99:
    print(
      '[Step %d] : Loss %.3f && Accuracy: %d%%' %
      (i + 1, loss / 100, num_correct)
    )
    loss = 0
    num_correct = 0   

  l, acc = train(im, label)
  loss += l
  num_correct += acc

BackPropagation
[Step 100] : Loss 2.066 && Accuracy: 21%
[Step 200] : Loss 1.941 && Accuracy: 35%
[Step 300] : Loss 1.901 && Accuracy: 32%
[Step 400] : Loss 1.726 && Accuracy: 46%
[Step 500] : Loss 1.551 && Accuracy: 52%
[Step 600] : Loss 1.502 && Accuracy: 60%
[Step 700] : Loss 1.146 && Accuracy: 67%
[Step 800] : Loss 1.188 && Accuracy: 59%
[Step 900] : Loss 1.097 && Accuracy: 64%
[Step 1000] : Loss 1.012 && Accuracy: 71%
[Step 1100] : Loss 0.898 && Accuracy: 77%
